In [21]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
#로컬 작업이므로 주석처리

Mounted at /content/drive


In [7]:
#filename ='/content/drive/MyDrive/데잇걸즈-공간분석 공모전/데이터/노후주택 목록 통합.xlsx'

In [22]:
df = pd.read_excel('old house.xlsx') #로컬 작업이므로 파일명 변경 요망

In [23]:
df['주소'] = df['시도']+" "+df['시군구']+" "+df['읍면동']+" "+df['상세주소']
df

,시도,시군구,읍면동,상세주소,특수지명,블록,로트,건물명,동명,통계용도,대지면적(㎡),건축면적(㎡),연면적(㎡),지하층수(층),지상층수(층),허가일자,착공일자,사용승인일자,동용도,주소
0,서울특별시,강북구,미아동,791-2175,NaN,NaN,NaN,NaN,NaN,주거용 > 단독주택 > 다가구주택,241.00,96.57,388.24,1,4,1986-07-01,NaN,1986-12-01,단독주택,서울특별시 강북구 미아동 791-2175
1,서울특별시,강북구,미아동,791-2193,NaN,NaN,NaN,NaN,NaN,주거용 > 단독주택 > 다가구주택,0.00,52.92,166.29,1,2,1990-01-25,NaN,1990-09-14,단독주택,서울특별시 강북구 미아동 791-2193
2,서울특별시,강북구,미아동,791-2195,NaN,NaN,NaN,NaN,NaN,주거용 > 단독주택 > 다가구주택,0.00,60.35,198.59,1,3,NaN,NaN,1983-11-24,단독주택,서울특별시 강북구 미아동 791-2195
3,서울특별시,강북구,미아동,791-2202,NaN,NaN,NaN,NaN,NaN,주거용 > 단독주택 > 다가구주택,0.00,105.70,318.12,1,2,1991-01-23,NaN,1991-11-18,단독주택,서울특별시 강북구 미아동 791-2202
4,서울특별시,강북구,미아동,791-2259,NaN,NaN,NaN,NaN,NaN,주거용 > 단독주택 > 다가구주택,99.09,58.98,167.90,1,2,1992-04-30,1992-06-26,1992-09-09,단독주택,서울특별시 강북구 미아동 791-2259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9547,서울특별시,강서구,화곡동,342-92,NaN,NaN,NaN,NaN,NaN,주거용 > 단독주택 > 다가구주택,190.00,72.18,211.30,1,2,1990-02-21,NaN,1990-09-29,단독주택,서울특별시 강서구 화곡동 342-92
9548,서울특별시,강서구,염창동,253-16,NaN,NaN,NaN,NaN,NaN,주거용 > 공동주택 > 다세대주택,232.20,98.34,328.40,1,3,1988-10-07,1988,1989-04-04,공동주택,서울특별시 강서구 염창동 253-16
9549,서울특별시,강서구,화곡동,56-507,NaN,NaN,NaN,NaN,NaN,주거용 > 공동주택 > 다세대주택,185.00,99.36,386.64,1,3,1991-01-30,1991-02-07,1991-07-05,공동주택,서울특별시 강서구 화곡동 56-507
9550,서울특별시,강서구,화곡동,56-267,NaN,NaN,NaN,NaN,NaN,주거용 > 단독주택 > 다가구주택,169.00,97.15,263.50,1,2,1991-05-23,1991-09-25,1992-02-28,단독주택,서울특별시 강서구 화곡동 56-267


In [24]:
# 카카오 API로 주소->위도, 경도 변환
import requests, json

def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  headers = {"Authorization": "KakaoAK API키"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}
  address_name = address['address_name']

  return crd



In [5]:
# 적용 확인하기
# crd=get_location("서울특별시 강서구 화곡동 1010-20")
# print(crd)

{'lat': '37.546684120058', 'lng': '126.834812326813'}


In [25]:
#코드 구동 시간을 알기 위해 time 패키지 import
import time

# for문으로 돌려서 각각의 주소값을 위도 경도로 변환해서 데이터프레임으로 생성하기
df_geo=pd.DataFrame(columns=['lat','lng'])

t1 = time.time()
for address in df['주소'] :
    #print(address)
    t2 = time.time()
    crd = get_location(address)
    lat = crd.get('lat')
    lng = crd.get('lng')
    df=df_geo.append(pd.DataFrame([[lat,lng]],columns=['lat','lng']))
    # if address < df['주소'][5]:
    #     print(time.time() - t2) #인덱스별 구동 시간


print(time.time() - t1) #지오코딩 총 구동 시간    




C:\Users\soyoung\AppData\Local\Temp\ipykernel_14112\3564483596.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df_geo.append(pd.DataFrame([[lat,lng]],columns=['lat','lng']))
C:\Users\soyoung\AppData\Local\Temp\ipykernel_14112\3564483596.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df_geo.append(pd.DataFrame([[lat,lng]],columns=['lat','lng']))
C:\Users\soyoung\AppData\Local\Temp\ipykernel_14112\3564483596.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df_geo.append(pd.DataFrame([[lat,lng]],columns=['lat','lng']))
C:\Users\soyoung\AppData\Local\Temp\ipykernel_14112\3564483596.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

ConnectTimeout: HTTPSConnectionPool(host='dapi.kakao.com', port=443): Max retries exceeded with url: /v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EA%B0%95%EB%B6%81%EA%B5%AC%20%EC%88%98%EC%9C%A0%EB%8F%99%20535-1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B56ECCDBA0>, 'Connection to dapi.kakao.com timed out. (connect timeout=None)'))

In [ ]:
print(df)

In [ ]:
# 엑셀 파일로 저장(코랩버전)
#excel_file.save('/content/drive/My Drive/본인폴더)